In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
spark_session = SparkSession.builder.appName('simple_statistics').getOrCreate()

Loading the data

In [3]:
df = spark_session.read.csv('data/StudentsPerformance.csv', header = True)

In [4]:
df.columns

['gender',
 'race/ethnicity',
 'parental level of education',
 'lunch',
 'test preparation course',
 'math score',
 'reading score',
 'writing score']

Categorical variables

In [5]:
df.select('gender').distinct().collect()

[Row(gender='female'), Row(gender='male')]

In [6]:
df.select('race/ethnicity').distinct().collect()

[Row(race/ethnicity='group B'),
 Row(race/ethnicity='group C'),
 Row(race/ethnicity='group D'),
 Row(race/ethnicity='group A'),
 Row(race/ethnicity='group E')]

In [7]:
df.select('parental level of education').distinct().collect()

[Row(parental level of education='some high school'),
 Row(parental level of education="associate's degree"),
 Row(parental level of education='high school'),
 Row(parental level of education="bachelor's degree"),
 Row(parental level of education="master's degree"),
 Row(parental level of education='some college')]

In [8]:
df.select('lunch').distinct().collect()

[Row(lunch='free/reduced'), Row(lunch='standard')]

In [9]:
df.select('test preparation course').distinct().collect()

[Row(test preparation course='completed'), Row(test preparation course='none')]

Numerical variables

In [10]:
math = df.select(df['math score'].cast('float').alias('math'))
mean = math.select(F.mean(math.math).alias('mean')).collect()[0]['mean']
std = math.select(F.stddev(math.math).alias('mean')).collect()[0]['mean']
min_ = math.select(F.min(math.math).alias('min')).collect()[0]['min']
max_ = math.select(F.max(math.math).alias('max')).collect()[0]['max']
print('Math: mean = ' + str(mean) + ', std = ' + str(std) + ', min = ' + str(min_) + ', max = ' + str(max_))

Math: mean = 66.089, std = 15.163080096009454, min = 0.0, max = 100.0


In [11]:
reading = df.select(df['reading score'].cast('float').alias('reading'))
mean = reading.select(F.mean(reading.reading).alias('mean')).collect()[0]['mean']
std = reading.select(F.stddev(reading.reading).alias('mean')).collect()[0]['mean']
min_ = reading.select(F.min(reading.reading).alias('min')).collect()[0]['min']
max_ = reading.select(F.max(reading.reading).alias('max')).collect()[0]['max']
print('Reading: mean = ' + str(mean) + ', std = ' + str(std) + ', min = ' + str(min_) + ', max = ' + str(max_))

Reading: mean = 69.169, std = 14.600191937252223, min = 17.0, max = 100.0


In [12]:
writing = df.select(df['writing score'].cast('float').alias('writing'))
mean = writing.select(F.mean(writing.writing).alias('mean')).collect()[0]['mean']
std = writing.select(F.stddev(writing.writing).alias('mean')).collect()[0]['mean']
min_ = writing.select(F.min(writing.writing).alias('min')).collect()[0]['min']
max_ = writing.select(F.max(writing.writing).alias('max')).collect()[0]['max']
print('Writing: mean = ' + str(mean) + ', std = ' + str(std) + ', min = ' + str(min_) + ', max = ' + str(max_))

Writing: mean = 68.054, std = 15.19565701086966, min = 10.0, max = 100.0


## Transforming variables

Let's transform the categorical variables into numerical ones.

In [13]:
# First we create an autoincrement id column
df = df.withColumn('id', F.monotonically_increasing_id())

In [14]:
for c in ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']:
    categories = df.select(c).distinct().rdd.flatMap(lambda x: x).collect()
    categories = categories[:-1]

    exprs = [F.when(F.col(c) == category, 1).otherwise(0).alias(category)
             for category in categories]

    df = df.join(df.select('id', *exprs), on = 'id').drop(c)

In [15]:
df = df.drop('id')

In [16]:
df.show(10)

+----------+-------------+-------------+------+-------+-------+-------+-------+----------------+------------------+-----------+-----------------+---------------+------------+---------+
|math score|reading score|writing score|female|group B|group C|group D|group A|some high school|associate's degree|high school|bachelor's degree|master's degree|free/reduced|completed|
+----------+-------------+-------------+------+-------+-------+-------+-------+----------------+------------------+-----------+-----------------+---------------+------------+---------+
|        69|           54|           55|     0|      1|      0|      0|      0|               0|                 0|          0|                0|              0|           0|        0|
|        62|           70|           75|     1|      0|      0|      1|      0|               0|                 0|          0|                0|              1|           0|        0|
|        90|           90|           91|     1|      1|      0|      0|    

And now we normalise all the variables:

In [17]:
for c in df.columns:
    values = df.select(df[c].cast('float').alias('values'))
    mean = values.select(F.mean(values.values).alias('mean')).collect()[0]['mean']
    std = values.select(F.stddev(values.values).alias('mean')).collect()[0]['mean']
    
    normalise_udf = F.udf(lambda x: (float(x) - mean) / std, T.FloatType())
    df.withColumn(c, normalise_udf(c))

In [18]:
df.show[10]

TypeError: 'method' object is not subscriptable